In [1]:
import pandas as pd 
import numpy as np
import math
import os
import gc
import sys
import pickle
from tqdm import *
import time
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter, defaultdict
import itertools
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import KFold, RepeatedKFold,train_test_split,StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import re
import datetime
from utils import *
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
color = sns.color_palette()
sns.set(style="whitegrid")
%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [4]:
huafen = [[1,  2,  3,  4],  [5,  6,  7,  8],  [9, 10, 11, 12], [13, 14, 15, 16],
 [17,18, 19, 20], [21, 22, 23, 24], [25, 26, 27, 28], [29, 30, 31, 32, 33]]

In [5]:
sub = pd.read_csv('../data/template_submit_result.csv',parse_dates=['ts'])
col = sub.columns.tolist()
index = ['ts', 'wtid']
enum_col = ['var016', 'var020', 'var047']
bool_col = ['var053', 'var066']
double_col = [i for i in col if i not in index+enum_col+bool_col]

In [9]:
%%time
"""修改路径"""
path = '../data/dataset/'
df = pd.DataFrame()
for j in tqdm(huafen):
    all_data = pd.DataFrame()
    sub_kk = pd.DataFrame()
    for i in j:
        f_path = path + str(i).zfill(3) + '/201807.csv'
        wtid_v = int(i)
        data = pd.read_csv(f_path,parse_dates=['ts'])

        sub_1 = sub[sub['wtid'] == wtid_v]
        ts_v = list(set(data['ts'].values)&set(sub_1['ts'].values))
        submit = sub_1[~sub_1['ts'].isin(ts_v)]
        data = pd.concat([data,submit],axis=0)
        data = data.sort_values(by=['wtid', 'ts']).reset_index(drop=True)
        all_data = all_data.append(data)
        sub_kk = sub_kk.append(sub_1)
    all_data = all_data.sort_values(by=['wtid', 'ts']).reset_index(drop=True)
    all_data['row_nulls'] = all_data.isnull().sum(axis=1)
    for i in ['day', 'week', 'hour', 'minute', 'second']:
        all_data[i] = time_extact(all_data['ts'],i)
    all_data['T_rank'] = all_data['minute'] + all_data['hour']*60
    all_data['S_rank'] = all_data['minute']*60 + all_data['second']
    train = all_data[all_data['row_nulls'].isin([0])]
    test = all_data[all_data['row_nulls'].isin([8])]
    fe_col = test[test.notnull()].columns.tolist()
    fe_col.remove('ts')
    model_fe =  ['var004', 'var042', 'var046', 'var060']
    for k in model_fe:
        lgb_model = lgb.LGBMRegressor(learning_rate=0.1, num_leaves=2**9, n_estimators=200, random_state=99, n_jobs=6)#167
        lgb_model.fit(train[fe_col], train[k])
        all_data.loc[test.index, k] = lgb_model.predict(test[fe_col])
    model_fe =  ['var027', 'var043', 'var061', 'var068']
#     train = all_data[all_data['row_nulls'].isin([0,8])]
#     test = all_data[all_data['row_nulls'].isin([19,27])]
    train = all_data[all_data['row_nulls'].isin([0])]
    test = all_data[all_data['row_nulls'].isin([19])]
    fe_col = test[test.notnull()].columns.tolist()
    fe_col.remove('ts')
    for k in model_fe:
        lgb_model = lgb.LGBMRegressor(num_leaves=2**9, n_estimators=187, random_state=99, n_jobs=6)#187
        lgb_model.fit(train[fe_col], train[k])
        all_data.loc[test.index, k] = lgb_model.predict(test[fe_col])
    for i in ['var008', 'var016', 'var020', 'var047', 'var066']:
        all_data[i] = 0
    for i in ['var001', 'var005', 'var038', 'var053', 'var060', 'var061']:
        all_data[i] = all_data[i].interpolate(method='linear')
    ddf = ['var002', 'var003', 'var006', 'var007', 'var011', 'var012', 
     'var014', 'var018', 'var021', 'var022', 'var024', 'var028', 
     'var029', 'var030', 'var033', 'var035', 'var036', 'var037', 'var040', 
     'var045', 'var051', 'var052', 'var055', 'var057', 'var062', 'var067']
    for i in ddf:
        all_data[i] = all_data[i].interpolate(method='linear')
    all_data['var053'] = all_data['var053'].abs().round()
    all_data = all_data.interpolate(method='nearest', axis=0)
    """值域外截断"""
    all_data['var060'] = all_data['var060'].map(lambda x: 50.04 if x > 50.04 else x)
    all_data['var060'] = all_data['var060'].map(lambda x: 49.90 if x<49.90 else x)
    all_data['var043'] = all_data['var043'].map(lambda x: 0 if x < 0.3 else x)
    
    sub_kk = sub_kk[['wtid','ts']].merge(all_data,on=['wtid','ts'],how='left')
    df = df.append(sub_kk)
    del all_data;gc.collect()
print('end shape: {}'.format(df.shape))
print('nan max number: {}'.format(df.isnull().sum().max()))

In [8]:
sub = sub[['ts','wtid']].merge(df, on=['ts','wtid'], how='left')

In [9]:
sub = sub[col]

In [10]:
for i in ['var016', 'var020', 'var047', 'var053', 'var066']:
    sub[i] = sub[i].astype('int32')
for col in sub.columns[2:]:
    sub[col] = sub[col].round(2)

In [11]:
pink_1 = pd.read_csv('../data/submit/pink_1.csv', index=False)
"""融合替换"""
asd = ['var027', 'var068', 'var004', 'var042', 'var046', 'var060', 'var043', 'var061']
for i in asd:
    pink_1[i] = sub[i].values
pink_1.to_csv('../data/submit/pink_2.csv',index=False)